### Calculate the luminosity during each segment

$L_{\gamma} = 4\pi D_L ^2 \int_{E_{min}}^{E_{max}} E \dfrac{dN}{dE} dE$

$D_L = 134.1$ $Mpc$

In [24]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy import units as u
from scipy import interpolate
from scipy.integrate import quad

In [25]:
emin = 0.1 * u.GeV
emax = 300 * u.GeV
D_l = 134.1 * u.Mpc
D_l = D_l.to(u.cm)
# emin = emin.to(u.erg)
# emax = emax.to(u.erg)

In [26]:
def e_dnde(e, n0, gamma, e0=1):
    return e * n0 * ((e / e0) ** (-gamma))    # Simple power law
    

dirs = ["1-20150918-20160527",  "2-20160913-20170615",  "3A-20170919-20180123",  "3B-20180206-20180622",  "4-20180918-20190528",  "5-20190928-20200608",  "6-20200915-20210503",  "7-20211101-20220619",  "all-20150918-20220619"]


In [27]:
dir = '3B-20180206-20180622/'

# lc_file_path = 'Output/Light_curve_001/4fgl_j0854.8+2006_lightcurve.fits'
lc_file_path = 'Output/Results.fits'
hdul = fits.open('./' + dir+lc_file_path)
lc = hdul[1].data
Table(lc).colnames

['Source_Name',
 'name',
 'class',
 'SpectrumType',
 'SpatialType',
 'SourceType',
 'SpatialModel',
 'RAJ2000',
 'DEJ2000',
 'GLON',
 'GLAT',
 'ts',
 'loglike',
 'npred',
 'offset',
 'offset_ra',
 'offset_dec',
 'offset_glon',
 'offset_glat',
 'offset_roi_edge',
 'pivot_energy',
 'flux_scan',
 'eflux_scan',
 'norm_scan',
 'dloglike_scan',
 'loglike_scan',
 'SpatialWidth',
 'Spatial_Filename',
 'Spectrum_Filename',
 'dec',
 'dec_err',
 'dnde',
 'dnde100',
 'dnde1000',
 'dnde10000',
 'dnde10000_err',
 'dnde10000_index',
 'dnde1000_err',
 'dnde1000_index',
 'dnde100_err',
 'dnde100_index',
 'dnde_err',
 'dnde_index',
 'eflux',
 'eflux100',
 'eflux1000',
 'eflux10000',
 'eflux10000_err',
 'eflux10000_ul95',
 'eflux1000_err',
 'eflux1000_ul95',
 'eflux100_err',
 'eflux100_ul95',
 'eflux_err',
 'eflux_ul95',
 'flux',
 'flux100',
 'flux1000',
 'flux10000',
 'flux10000_err',
 'flux10000_ul95',
 'flux1000_err',
 'flux1000_ul95',
 'flux100_err',
 'flux100_ul95',
 'flux_err',
 'flux_ul95',
 'glat

In [28]:
lc['pivot_energy']

array([  803.71717331,  1361.17375515,  2220.97573293,  2082.98891247,
        8610.73540588,   286.94936635,   474.89264567,  2784.0178991 ,
        5575.70288463,  2542.86320698,  2162.20417284,   591.32895184,
         853.58643422,  3302.17152629,  1199.58539835,   981.6605098 ,
        8664.20225248, 22912.98299322,  1372.2302825 ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
         840.95921713,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
        1000.        ,  1000.        ,  1000.        ,  1000.        ,
      

In [29]:
Table(lc)['Source_Name', 'pivot_energy']

Source_Name,pivot_energy
str48,float64
4FGL J0854.8+2006,803.7171733140149
4FGL J0856.8+2056,1361.1737551515728
4FGL J0902.4+2051,2220.975732928973
4FGL J0839.4+1803,2082.988912470736
4FGL J0908.9+2311,8610.735405875812
4FGL J0910.6+2247,286.9493663525024
4FGL J0836.2+2141,474.8926456701408
4FGL J0912.5+1556,2784.017899103264
4FGL J0831.5+1747,5575.702884631161


In [30]:
params = pd.read_csv('param.csv', index_col=0, sep='&', header=0)

In [31]:
params

,TS,Flux,Flux_err,eFlux,eFlux_err,log_N0,log_N0_err,g,g_err
Seg,,,,,,,,,
1,1586.61,10.79,0.70,0.46,0.03,-9.51,0.02,2.14,0.04
2,891.89,4.07,0.05,0.35,0.01,-9.94,0.05,1.92,0.05
3A,155.29,3.44,0.95,0.15,0.03,-9.89,0.16,2.19,0.19
3B,154.60,3.88,0.67,0.19,0.03,-10.06,0.12,1.97,0.12
4,263.04,4.80,0.78,0.20,0.03,-9.91,0.06,2.21,0.10
5,379.65,4.52,0.72,0.26,0.05,-9.96,0.05,2.10,0.07
6,308.48,3.72,0.67,0.21,0.03,-9.97,0.06,2.03,0.08
7,320.09,2.06,0.25,0.19,0.03,-10.20,0.13,1.84,0.12
all,4774.88,5.18,0.05,0.27,0.04,-9.86,0.01,2.12,0.02


In [32]:
luminosity = []
for i in range(0, len(dirs)):
    idx = dirs[i].split('-')[0]
    print(idx)
    lc_file_path = 'Output/Results.fits'
    hdul = fits.open('./' + dirs[i]+'/'+lc_file_path)
    lc = hdul[1].data
    e0 = lc['pivot_energy'][0] * u.MeV
    n0 = 10 ** (params['log_N0'][idx]) * (1/(u.cm**2 * u.s * u.MeV))
    gamma = params['g'][idx]
    # print(n0, gamma, e0)
    res = 4*np.pi * (D_l**2) * quad(e_dnde, emin.value, emax.value, args=(n0.to(1/(u.cm**2 * u.s * u.GeV)).value, gamma, e0.to(u.GeV).value), epsabs=1e-18, epsrel=1e-18, limit=100000) * u.GeV/(u.cm**2 * u.s)
    print([res.to(u.erg/u.s).value[0], res.value[0]])
    # print(res)
    luminosity.append([res.to(u.erg/u.s).value[0], res.value[0]])


1
[2.693323142452543e+45, 1.6810400834072727e+48]
2
[1.9685859519831183e+45, 1.2286947083158615e+48]
3A
[1.965589985995994e+45, 1.2268247734263201e+48]
3B
[1.6488005028944112e+45, 1.0291003300790939e+48]
4
[6.996285918298251e+44, 4.366738204658059e+47]
5
[2.2079275010822094e+45, 1.3780799533756087e+48]
6
[1.552729422348346e+45, 9.691374779769421e+47]
7
[2.822528093501738e+45, 1.7616834708511537e+48]
all
[1.5911278537902065e+45, 9.931038938058852e+47]


In [33]:
dirs[0]

'1-20150918-20160527'

In [34]:
final_table = pd.read_csv('final_table.tex', index_col=0, sep = ' ', header=None)

In [35]:
final_table

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
0,,,,,,,,,,,,,,,,,,,,,
1,&,1586.61,&,10.79,$\pm$,0.70,&,0.46,$\pm$,0.03,...,-9.51,$\pm$,0.02,&,2.14,$\pm$,0.04,&,2.69,\\
2,&,891.89,&,4.07,$\pm$,0.05,&,0.35,$\pm$,0.01,...,-9.94,$\pm$,0.05,&,1.92,$\pm$,0.05,&,1.97,\\
3A,&,155.29,&,3.44,$\pm$,0.95,&,0.15,$\pm$,0.03,...,-9.89,$\pm$,0.16,&,2.19,$\pm$,0.19,&,1.97,\\
3B,&,154.60,&,3.88,$\pm$,0.67,&,0.19,$\pm$,0.03,...,-10.06,$\pm$,0.12,&,1.97,$\pm$,0.12,&,1.65,\\
4,&,263.04,&,4.80,$\pm$,0.78,&,0.20,$\pm$,0.03,...,-9.91,$\pm$,0.06,&,2.21,$\pm$,0.10,&,0.70,\\
5,&,379.65,&,4.52,$\pm$,0.72,&,0.26,$\pm$,0.05,...,-9.96,$\pm$,0.05,&,2.10,$\pm$,0.07,&,2.21,\\
6,&,308.48,&,3.72,$\pm$,0.67,&,0.21,$\pm$,0.03,...,-9.97,$\pm$,0.06,&,2.03,$\pm$,0.08,&,1.55,\\
7,&,320.09,&,2.06,$\pm$,0.25,&,0.19,$\pm$,0.03,...,-10.20,$\pm$,0.13,&,1.84,$\pm$,0.12,&,2.82,\\
all,&,4774.88,&,5.18,$\pm$,0.05,&,0.27,$\pm$,0.04,...,-9.86,$\pm$,0.01,&,2.12,$\pm$,0.02,&,1.59,\\


In [14]:
final_table[19] = [' & '] * len(final_table)
final_table[20] = [np.round(luminosity[i][0]/1e45, 2) for i in range(0, len(dirs))]
# final_table[21] = [' & '] * len(final_table)
# final_table[22] = [np.round(luminosity[i][1]/1e48, 2) for i in range(0, len(dirs)-1)]
final_table[21] = [' \\\\'] * len(final_table)
# final_table.drop(columns=[22, 23], inplace=True)

In [15]:
final_table

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
0,,,,,,,,,,,,,,,,,,,,,
1,&,1586.61,&,10.79,$\pm$,0.70,&,0.46,$\pm$,0.03,...,-9.51,$\pm$,0.02,&,2.14,$\pm$,0.04,&,2.69,\\
2,&,891.89,&,4.07,$\pm$,0.05,&,0.35,$\pm$,0.01,...,-9.94,$\pm$,0.05,&,1.92,$\pm$,0.05,&,1.97,\\
3A,&,155.29,&,3.44,$\pm$,0.95,&,0.15,$\pm$,0.03,...,-9.89,$\pm$,0.16,&,2.19,$\pm$,0.19,&,1.97,\\
3B,&,154.60,&,3.88,$\pm$,0.67,&,0.19,$\pm$,0.03,...,-10.06,$\pm$,0.12,&,1.97,$\pm$,0.12,&,1.65,\\
4,&,263.04,&,4.80,$\pm$,0.78,&,0.20,$\pm$,0.03,...,-9.91,$\pm$,0.06,&,2.21,$\pm$,0.10,&,0.70,\\
5,&,379.65,&,4.52,$\pm$,0.72,&,0.26,$\pm$,0.05,...,-9.96,$\pm$,0.05,&,2.10,$\pm$,0.07,&,2.21,\\
6,&,308.48,&,3.72,$\pm$,0.67,&,0.21,$\pm$,0.03,...,-9.97,$\pm$,0.06,&,2.03,$\pm$,0.08,&,1.55,\\
7,&,320.09,&,2.06,$\pm$,0.25,&,0.19,$\pm$,0.03,...,-10.20,$\pm$,0.13,&,1.84,$\pm$,0.12,&,2.82,\\
all,&,4774.88,&,5.18,$\pm$,0.05,&,0.27,$\pm$,0.04,...,-9.86,$\pm$,0.01,&,2.12,$\pm$,0.02,&,1.59,\\


In [16]:
final_table.to_csv('final_table.tex', sep=' ', header=None)